In [ ]:
TOKNIZER ="cyberagent/open-calm-7b"
CAUSAL_LM="cyberagent/open-calm-7b"

from configs.finetune_config import FinetuneConfig
conf_path="/home/user0/work/data/config/default.yaml"
conf: FinetuneConfig = FinetuneConfig.load(conf_path)

In [ ]:
import torch
from peft import PeftModel
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    GPTNeoXTokenizerFast,
    BitsAndBytesConfig,
)

compute_dtype = torch.bfloat16
quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_quant_type="nf4", # {'fp4', 'nf4'}
)

# トークナイザーとモデルの準備
try:
    tokenizer = AutoTokenizer.from_pretrained(
        conf.base_model,
        use_fast=False,
        cache_dir=conf.model_cache_dir,
    )
except ValueError as e:
    model_config = AutoConfig.from_pretrained(conf.base_model)
    if model_config.model_type == "gpt_neox":
        tokenizer = GPTNeoXTokenizerFast.from_pretrained(
            conf.base_model,
            use_fast=False,
            cache_dir=conf.model_cache_dir,
        )
    else:
        raisetokenizer = AutoTokenizer.from_pretrained(
)
model = AutoModelForCausalLM.from_pretrained(
    CAUSAL_LM,
    device_map="auto", 
    torch_dtype=compute_dtype,
    quantization_config=quantization_config,
    cache_dir=conf.model_cache_dir,
)
model = PeftModel.from_pretrained(model, conf.output_dir)

In [ ]:
from utils.prompter import Prompter
prompter:Prompter = Prompter(conf.prompt_template_name, conf.prompt_template_dir_path)
def generate_prompt(prompt: str):
    return prompter.generate_prompt(prompt)

In [ ]:
# prompt_prefix="ユーザー: あなたは『アルパ』という女の子になって受け答えをしてください。<NL>"
def talk(prompt):
    prompt = generate_prompt(prompt)
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    with torch.no_grad():
      # output_ids = model.generate(
      #     # token_ids.to(model.device),
      #     token_ids,
      #     do_sample=True,
      #     max_new_tokens=64,
      #     temperature=0.7,
      #     pad_token_id=tokenizer.pad_token_id,
      #     bos_token_id=tokenizer.bos_token_id,
      #     eos_token_id=tokenizer.eos_token_id)
      output_ids = model.generate(
        input_ids=token_ids,
        max_new_tokens=64,
        do_sample=True,
        temperature=0.7,
        top_p=0.75,
        top_k=40,
        no_repeat_ngram_size=2,
      )
    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
    output = output.replace("<NL>", "\n")
    return output

In [ ]:
talk("こんにちは！今日は暑かったね。")